# December 23, 2023

https://adventofcode.com/2023/day/23

In [89]:
text = f'''#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#'''

test_text = text.split("\n")

In [90]:
fn = "data/23.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz_text = [x.strip() for x in text]

In [142]:
class Forest:
    def __init__( self, text ):
        self.text = text
        
        self.width = len(text[0])
        self.height = len(text)
        self.start = [0,1]
        self.goal = [self.height-1, self.width-2]

    def symbol( self, rc ):
        '''return symbol on map at location rc=[row,col]'''
        r,c = rc
        if (c<0 or c==self.width or c<0 or c==self.height):
            return None
        return self.text[r][c]
    
    def slope_okay( self, rc0, rc1 ):
        '''
        check if we can go from rc0 to rc1 if the slope at rc1 is slippery
        
        assumes rc0 is non-diagonally adjacent to rc1
        '''
        sym = self.symbol( rc1 )
        
        if sym not in ["<", ">", "v", "^"]:
            raise BaseException(f'''{rc1[0]}, {rc1[1]} is not a slope''')

        # going up
        if rc1[0] == rc0[0] - 1:
            return sym != "v"
        # going down
        if rc1[0] == rc0[0] + 1:
            return sym != "^"
        # going left
        if rc1[1] == rc0[1] - 1:
            return sym != ">"
        # going right
        if rc1[1] == rc0[1] + 1:
            return sym != "<"
        

    
    def find_neighbors( self, rc ):
        '''return list of neighbors for the given point'''
        r,c = rc

        radj_list = [-1, 0, 0, +1]
        cadj_list = [0, -1, +1, 0]
        nbrs = []

        for radj, cadj in zip(radj_list, cadj_list):
            look = [r+radj, c+cadj]
            sym = self.symbol( look )
            if (sym is not None) and (sym != "#"):
                nbrs.append( look )

        return nbrs

    def measure_segment( self, cur, prev ):
        '''
        count the number of steps from cur to either the goal or the next decision point
        
        returns step count, end of the segment, and list of neighbors for next move
        '''
        steps = 0
        while True:
            # find open spaces, excluding the previous space we were in
            nbrs = self.find_neighbors(cur)
            nbrs = [ n for n in nbrs if n != prev ]
            if len(nbrs) > 1:
                return steps, cur, nbrs
            
            steps += 1
            prev = cur
            cur = nbrs[0]
            if cur == self.goal:
                return steps, cur, []
    
    def max_dist( self, cur=None, prev=None, slippy=True, history=None ):
        '''return the maximum distance from cur to the goal
        
        if slippy, must follow direction of slopes <>v^
        otherwise, free to go either direction there
        '''
        if cur is None:
            cur = self.start
        if prev is None:
            prev = [0,0]
        if history is None:
            history = [cur]
        else:
            history = history.copy()
            history.append(cur)
            
        steps_so_far, cur, nbrs = self.measure_segment( cur, prev )

        # We made it, return the answer
        if cur == self.goal:
            return steps_so_far

        # oops, we landed back at a previous spot.
        # Let calling function know this path doesn't work!
        if cur in history:
            return None
        
        history.append(cur)
        
        # find possible paths forward
        to_try = []
        for n in nbrs:
            # We can't go this direction
            if n in history:
                continue

            if (not slippy or self.slope_okay(cur, n)):
                to_try.append(n)

        # try all those directions and find the best one
        res = [ self.max_dist(cur=t, prev=cur, slippy=slippy, history=history) for t in to_try ]
        res = [r for r in res if r is not None]
        if len(res) == 0:
            return None
        
        return steps_so_far + max(res) + 1

### Part 1

In [143]:
test = Forest(test_text)
test.text

['#.#####################',
 '#.......#########...###',
 '#######.#########.#.###',
 '###.....#.>.>.###.#.###',
 '###v#####.#v#.###.#.###',
 '###.>...#.#.#.....#...#',
 '###v###.#.#.#########.#',
 '###...#.#.#.......#...#',
 '#####.#.#.#######.#.###',
 '#.....#.#.#.......#...#',
 '#.#####.#.#.#########v#',
 '#.#...#...#...###...>.#',
 '#.#.#v#######v###.###v#',
 '#...#.>.#...>.>.#.###.#',
 '#####v#.#.###v#.#.###.#',
 '#.....#...#...#.#.#...#',
 '#.#########.###.#.#.###',
 '#...###...#...#...#.###',
 '###.###.#.###v#####v###',
 '#...#...#.#.>.>.#.>.###',
 '#.###.###.#.###.#.#v###',
 '#.....###...###...#...#',
 '#####################.#']

In [144]:
test.measure_segment( [0,1], [0,0] )

(15, [5, 3], [[5, 4], [6, 3]])

In [145]:
test.max_dist(slippy=True)

94

In [146]:
puzz = Forest(puzz_text)

In [147]:
puzz.max_dist()

2134

### Part 2

In [148]:
test.max_dist(slippy=False)

154

In [149]:
puzz.max_dist(slippy=False)

KeyboardInterrupt: 